In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/raw/items_titles.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [4]:
df_test = pd.read_csv("../data/raw/items_titles_test.csv")

In [ ]:
df_test.head()

In [ ]:
df_test.tail()

In [ ]:
df['desc'] = df['ITE_ITEM_TITLE']
df_test['desc'] = df_test['ITE_ITEM_TITLE']

In [ ]:
df_join = df_test.merge(df, on="ITE_ITEM_TITLE", how="left")

In [ ]:
df_join.head()

In [ ]:
df_join.shape

In [ ]:
df_join[df_join["desc_y"].isna()].shape

In [ ]:
df_join[~df_join["desc_y"].isna()].shape

In [ ]:
df_join[~df_join["desc_y"].isna()]

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf_idf = TfidfVectorizer()
df_test_tfidf = tf_idf.fit_transform(df_test)

In [ ]:
df_tfidf = tf_idf.transform(df)

In [ ]:
tf_idf.get_feature_names_out()

In [ ]:
# !pip install sentence-transformers

In [6]:
from sentence_transformers import SentenceTransformer, util

/Users/adrianofreitas/prj/meli/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = SentenceTransformer('stsb-roberta-large')

In [32]:
sentences = [sentence.lower() for sentence in df["ITE_ITEM_TITLE"]]

In [33]:
sentences_test = [sentence.lower() for sentence in df_test["ITE_ITEM_TITLE"]]

In [34]:
%%time
embeddings = model.encode(sentences)

CPU times: user 1h 21min 8s, sys: 2min 16s, total: 1h 23min 24s
Wall time: 49min 43s


In [35]:
%%time
embeddings_test = model.encode(sentences_test)

CPU times: user 26min 41s, sys: 34.3 s, total: 27min 15s
Wall time: 14min 17s


In [36]:
%%time
for i, sent_test in enumerate(zip(sentences_test, embeddings_test)):
    # embed_sent = model.encode(sent)
    sent, embed_sent = sent_test
    cos_sim = util.cos_sim(embed_sent, embeddings)
    # if i == 0:
    #     print(cos_sim.shape, cos_sim, '\n')
    
    sent_sim = zip(sentences, cos_sim[0])
    # if i == 0:
    #     print(len(cos_sim), sent_sim, '\n')
    # for arr in cos_sim:
    #     for i, val in enumerate(arr):
    #         top_sim.append([sentences[i],val])

    top_sim = sorted(sent_sim, key=lambda x: x[1], reverse=True)[:2]
    # if i == 0:
    #     print(len(cos_sim), top_sim, '\n')
    # if i<3:
    #     for arr in top_sim:
    #         print(f'Base : {sent}')
    #         print(f'Score : {arr[1]}')
    #         print(f'Sentence : {arr[0]}\n')
    # print("\n\n")

CPU times: user 3h 6min 9s, sys: 3min 26s, total: 3h 9min 36s
Wall time: 2h 36min 11s


In [37]:
embeddings[0]

array([ 0.73466694, -0.5601898 ,  0.09044465, ...,  0.6118551 ,
       -1.1363635 ,  0.9054669 ], dtype=float32)